In [1]:
# Instalar e importar pacotes
import pandas as pd
import numpy as np
import glob
import os
import unicodedata
import re
import io

from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***AJUSTE DADOS CASOS***

In [2]:
# Definir caminho de pasta
caminho_pasta = "/content/drive/MyDrive/Leptospirose/SINAN/*.csv"

# Listar arquivos contidos nessa pasta
arquivos_casos = glob.glob(caminho_pasta)

print(f"Total de arquivos encontrados: {len(arquivos_casos)}")
print("Arquivos identificados:")
for a in arquivos_casos:
    print(a)

Total de arquivos encontrados: 16
Arquivos identificados:
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2010.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2011.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2012.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2013.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2014.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2015.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2016.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2017.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2018.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2019.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2020.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2021.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2022.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2023.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2008.csv
/content/drive/MyDrive/Leptospirose/SINAN/SINAN_2009.csv


In [3]:
lista_casos = []
for arquivo in arquivos_casos:
    ano = os.path.basename(arquivo).split('_')[1].split('.')[0]
    df = pd.read_csv(arquivo, encoding='latin1', sep=',')
    df = df.melt(id_vars=['MES'], var_name='ESTADO', value_name='CASOS')
    df['ANO'] = int(ano)
    lista_casos.append(df)

casos = pd.concat(lista_casos, ignore_index=True)

In [4]:
casos

,MES,ESTADO,CASOS,ANO
0,Janeiro,RO,0,2010
1,Fevereiro,RO,0,2010
2,Marco,RO,2,2010
3,Abril,RO,1,2010
4,Maio,RO,3,2010
...,...,...,...,...
4795,Agosto,DF,3,2009
4796,Setembro,DF,3,2009
4797,Outubro,DF,1,2009
4798,Novembro,DF,2,2009


# ***AJUSTE DADOS CLIMA***

## Etapa 1 - Ajuste Geral do Banco

In [5]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2008")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_CO_DF_A001_BRASILIA_01-01-2008_A_31-12-2008.CSV → 8784 linhas, 20 colunas
INMET_CO_GO_A002_GOIANIA_01-01-2008_A_31-12-2008.CSV → 8784 linhas, 20 colunas
INMET_CO_MS_A702_CAMPO GRANDE_01-01-2008_A_31-12-2008.CSV → 8784 linhas, 20 colunas
INMET_CO_MT_A901_CUIABA_01-01-2008_A_31-12-2008.CSV → 8784 linhas, 20 colunas
INMET_NE_AL_A303_MACEIO_01-01-2008_A_31-12-2008.CSV → 8784 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [6]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2009")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_CO_DF_A001_BRASILIA_01-01-2009_A_31-12-2009.CSV → 8760 linhas, 20 colunas
INMET_CO_GO_A002_GOIANIA_01-01-2009_A_31-12-2009.CSV → 8760 linhas, 20 colunas
INMET_CO_MS_A702_CAMPO GRANDE_01-01-2009_A_31-12-2009.CSV → 8760 linhas, 20 colunas
INMET_CO_MT_A901_CUIABA_01-01-2009_A_31-12-2009.CSV → 8760 linhas, 20 colunas
INMET_NE_AL_A303_MACEIO_01-01-2009_A_31-12-2009.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [7]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2010")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2010.CSV → 8760 linhas, 20 colunas
INMET_AL_2010.CSV → 8760 linhas, 20 colunas
INMET_AM_2010.CSV → 8760 linhas, 20 colunas
INMET_BA_2010.CSV → 8760 linhas, 20 colunas
INMET_CE_2010.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [8]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2011")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2011.CSV → 8760 linhas, 20 colunas
INMET_AL_2011.CSV → 8760 linhas, 20 colunas
INMET_AM_2011.CSV → 8760 linhas, 20 colunas
INMET_BA_2011.CSV → 8760 linhas, 20 colunas
INMET_CE_2011.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [9]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2012")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2012.CSV → 8784 linhas, 20 colunas
INMET_AL_2012.CSV → 8784 linhas, 20 colunas
INMET_AM_2012.CSV → 8784 linhas, 20 colunas
INMET_BA_2012.CSV → 8784 linhas, 20 colunas
INMET_CE_2012.CSV → 8784 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [10]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2013")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2013.CSV → 8760 linhas, 20 colunas
INMET_AL_2013.CSV → 8760 linhas, 20 colunas
INMET_AM_2013.CSV → 8760 linhas, 20 colunas
INMET_BA_2013.CSV → 8760 linhas, 20 colunas
INMET_CE_2013.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [11]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2014")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2014.CSV → 8760 linhas, 20 colunas
INMET_AL_2014.CSV → 8760 linhas, 20 colunas
INMET_AM_2014.CSV → 8760 linhas, 20 colunas
INMET_BA_2014.CSV → 8760 linhas, 20 colunas
INMET_CE_2014.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [12]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2015")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2015.CSV → 8760 linhas, 20 colunas
INMET_AL_2015.CSV → 8760 linhas, 20 colunas
INMET_AM_2015.CSV → 8760 linhas, 20 colunas
INMET_BA_2015.CSV → 8760 linhas, 20 colunas
INMET_CE_2015.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [13]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2016")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2016.CSV → 8784 linhas, 20 colunas
INMET_AL_2016.CSV → 8784 linhas, 20 colunas
INMET_AM_2016.CSV → 8784 linhas, 20 colunas
INMET_BA_2016.CSV → 8784 linhas, 20 colunas
INMET_CE_2016.CSV → 8784 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [14]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2017")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2017.CSV → 8760 linhas, 20 colunas
INMET_AL_2017.CSV → 8760 linhas, 20 colunas
INMET_AM_2017.CSV → 8760 linhas, 20 colunas
INMET_BA_2017.CSV → 8760 linhas, 20 colunas
INMET_CE_2017.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [15]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2018")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2018.CSV → 8760 linhas, 20 colunas
INMET_AL_2018.CSV → 8760 linhas, 20 colunas
INMET_AM_2018.CSV → 8760 linhas, 20 colunas
INMET_BA_2018.CSV → 8760 linhas, 20 colunas
INMET_CE_2018.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [16]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2019")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2019.CSV → 8760 linhas, 20 colunas
INMET_AL_2019.CSV → 8760 linhas, 20 colunas
INMET_AM_2019.CSV → 8760 linhas, 20 colunas
INMET_BA_2019.CSV → 8760 linhas, 20 colunas
INMET_CE_2019.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [17]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2020")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2020.CSV → 8784 linhas, 20 colunas
INMET_AL_2020.CSV → 8784 linhas, 20 colunas
INMET_AM_2020.CSV → 8784 linhas, 20 colunas
INMET_BA_2020.CSV → 8784 linhas, 20 colunas
INMET_CE_2020.CSV → 8784 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [18]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2021")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2021.CSV → 8760 linhas, 20 colunas
INMET_AL_2021.CSV → 8760 linhas, 20 colunas
INMET_AM_2021.CSV → 8760 linhas, 20 colunas
INMET_BA_2021.CSV → 8760 linhas, 20 colunas
INMET_CE_2021.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [19]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2022")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2022.CSV → 8760 linhas, 20 colunas
INMET_AL_2022.CSV → 8760 linhas, 20 colunas
INMET_AM_2022.CSV → 8760 linhas, 20 colunas
INMET_BA_2022.CSV → 8760 linhas, 20 colunas
INMET_CE_2022.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


In [20]:
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET/2023")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
N_LINHAS_PULAR = 8
ENC = "latin1"

# COLETA DE ARQUIVOS
arquivos_csv = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f" Total de arquivos encontrados: {len(arquivos_csv)}\n")

# PROCESSAMENTO
ok = 0
erros = []

for arquivo in arquivos_csv:
    try:
        # LER ARQUIVO COMO TEXTO, SUBSTITUIR VÍRGULA POR PONTO
        with open(arquivo, 'r', encoding=ENC) as f:
            linhas = f.readlines()

        # Pular 8 primeiras linhas (cabeçalho)
        linhas_dados = linhas[N_LINHAS_PULAR:]

        # Substituir vírgula por ponto (EXCETO na linha de header)
        linhas_processadas = [linhas_dados[0]]

        for linha in linhas_dados[1:]:  # Dados
            # Substituir vírgula por ponto
            linha_corrigida = linha.replace(',', '.')
            linhas_processadas.append(linha_corrigida)

        # LER COM PANDAS (agora com pontos)
        conteudo = ''.join(linhas_processadas)
        df = pd.read_csv(
            io.StringIO(conteudo),
            sep=SEP,
            engine="python",
            on_bad_lines="skip"
        )

        # Criar estrutura de pastas
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # SALVAR (já com pontos)
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(
            arquivo_destino,
            sep=SEP,
            index=False,
            encoding=ENC
        )

        ok += 1
        if ok <= 5:
            print(f"{arquivo.name} → {df.shape[0]} linhas, {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"{arquivo.name}: {e}")

# RESUMO
print(f"\n{'='*80}")
print(f"ETAPA 1 CONCLUÍDA")
print(f"{'='*80}")
print(f"Processados: {ok}/{len(arquivos_csv)}")
print(f"Erros: {len(erros)}")

if erros:
    print(f"\n  Primeiros erros:")
    for nome, erro in erros[:10]:
        print(f"   • {nome}: {erro[:60]}")

 Total de arquivos encontrados: 25

INMET_AC_2023.CSV → 8760 linhas, 20 colunas
INMET_AL_2023.CSV → 8760 linhas, 20 colunas
INMET_AM_2023.CSV → 8760 linhas, 20 colunas
INMET_BA_2023.CSV → 8760 linhas, 20 colunas
INMET_CE_2023.CSV → 8760 linhas, 20 colunas

ETAPA 1 CONCLUÍDA
Processados: 25/25
Erros: 0


## Etapa 2: Renomear Colunas

In [21]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2008")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_CO_DF_A001_BRASILIA_01-01-2008_A_31-12-2008.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_CO_GO_A002_GOIANIA_01-01-2008_A_31-12-2008.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_CO_MS_A702_CAMPO GRANDE_01-01-2008_A_31-12-2008.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_CO_MT_A901_CUIABA_01-01-2008_A_31-12-2008.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
 

In [22]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2009")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_CO_DF_A001_BRASILIA_01-01-2009_A_31-12-2009.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CO_GO_A002_GOIANIA_01-01-2009_A_31-12-2009.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CO_MS_A702_CAMPO GRANDE_01-01-2009_A_31-12-2009.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CO_MT_A901_CUIABA_01-01-2009_A_31-12-2009.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
 

In [23]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2010")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2010.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2010.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2010.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2010.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2010.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [24]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2011")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2011.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2011.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2011.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2011.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2011.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [25]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2012")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2012.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_AL_2012.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_AM_2012.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_BA_2012.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_CE_2012.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [26]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2013")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2013.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2013.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2013.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2013.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2013.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [27]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2014")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2014.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2014.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2014.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2014.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2014.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [28]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2015")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2015.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2015.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2015.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2015.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2015.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [29]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2016")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2016.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_AL_2016.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_AM_2016.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_BA_2016.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_CE_2016.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [30]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2017")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2017.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2017.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2017.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2017.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2017.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [31]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2018")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2018.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2018.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2018.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2018.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2018.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [32]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2019")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2019.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2019.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2019.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2019.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2019.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [33]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2020")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2020.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 18
   Colunas DEPOIS: 18
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 18 colunas

📄 INMET_AL_2020.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_AM_2020.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_BA_2020.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8784 linhas × 19 colunas

📄 INMET_CE_2020.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [34]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2021")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2021.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 2
   Colunas DEPOIS: 2
   Nomes das colunas: ['DATA', 'HORA']...
   Salvo: 8760 linhas × 2 colunas

📄 INMET_AL_2021.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 14
   Colunas DEPOIS: 14
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 14 colunas

📄 INMET_AM_2021.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2021.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2021.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRES

In [35]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2022")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2022.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2022.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2022.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2022.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2022.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [36]:
# CONFIGURAÇÃO
PASTA_ORIGEM = Path("/content/drive/MyDrive/Leptospirose/INMET_LIMPO/2023")
PASTA_DESTINO = Path("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/")
PASTA_DESTINO.mkdir(exist_ok=True)

SEP = ";"
ENC = "latin1"
SENTINEL = -9999

# DICIONÁRIO DE RENOMEAÇÃO
# Mapeamento EXATO: nome_original → nome_novo

MAPEAMENTO_EXATO = {
    # Seus nomes EXATOS
    "DATA (YYYY-MM-DD)": "DATA",
    "HORA (UTC)": "HORA",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "PRECIPITACAO",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "PRESSAO",
    "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "PRESSAO_MAX",
    "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "PRESSAO_MIN",
    "RADIACAO GLOBAL (KJ/m²)": "RADIACAO",
    "RADIACAO GLOBAL (Kj/m²)": "RADIACAO",  # Variação com Kj minúsculo
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "TEMPERATURA",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "TEMPERATURA_ORVALHO",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "TEMP_MAX",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "TEMP_MIN",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MAX",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "TEMP_ORV_MIN",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": "UMIDADE_MAX",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": "UMIDADE_MIN",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "UMIDADE",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "VENTO_DIR",
    "VENTO, RAJADA MAXIMA (m/s)": "VENTO_RAJ",
    "VENTO, VELOCIDADE HORARIA (m/s)": "VENTO_VEL",

    # Possíveis variações (caso existam)
    "Data": "DATA",
    "Hora UTC": "HORA",
}

# PROCESSAMENTO
arquivos = sorted(PASTA_ORIGEM.rglob("*.CSV"))
print(f"Arquivos para processar: {len(arquivos)}\n")

ok = 0
erros = []

for arquivo in arquivos:
    try:
        # Ler arquivo
        df = pd.read_csv(arquivo, sep=SEP, encoding=ENC)

        print(f"\n📄 {arquivo.name}")
        print(f"   Colunas ANTES: {df.shape[1]}")

        # Remover Unnamed
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", regex=True, na=False)]
        df = df.dropna(axis=1, how='all')

        print(f"   Colunas depois de limpar: {df.shape[1]}")

        # RENOMEAR usando o dicionário
        df = df.rename(columns=MAPEAMENTO_EXATO)

        print(f"   Colunas DEPOIS: {df.shape[1]}")
        print(f"   Nomes das colunas: {df.columns.tolist()[:5]}...")

        # Trocar -9999 por NA
        num_cols = df.select_dtypes(include="number").columns
        if len(num_cols) > 0:
            df[num_cols] = df[num_cols].replace(SENTINEL, pd.NA)

        # Criar pasta de destino
        subpasta = arquivo.parent.name
        pasta_ano = PASTA_DESTINO / subpasta
        pasta_ano.mkdir(exist_ok=True)

        # Salvar
        arquivo_destino = pasta_ano / arquivo.name
        df.to_csv(arquivo_destino, sep=SEP, index=False, encoding=ENC)

        ok += 1
        print(f"   Salvo: {df.shape[0]} linhas × {df.shape[1]} colunas")

    except Exception as e:
        erros.append((arquivo.name, str(e)))
        print(f"   ERRO: {e}")

# ==================== RESUMO ====================
print(f"\n{'='*80}")
print(f"PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"  Processados: {ok}/{len(arquivos)}")
print(f" Erros: {len(erros)}")

if erros:
    print(f"\n Arquivos com erro:")
    for nome, msg in erros:
        print(f"   • {nome}: {msg[:80]}")

Arquivos para processar: 25


📄 INMET_AC_2023.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AL_2023.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_AM_2023.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_BA_2023.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das colunas: ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO', 'PRESSAO_MAX']...
   Salvo: 8760 linhas × 19 colunas

📄 INMET_CE_2023.CSV
   Colunas ANTES: 20
   Colunas depois de limpar: 19
   Colunas DEPOIS: 19
   Nomes das co

In [37]:
arquivos_clima = glob.glob("/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/**/*.CSV", recursive=True)

print(f"Total de arquivos encontrados: {len(arquivos_clima)}")
print("Arquivos identificados:")
for a in arquivos_clima:
    print(a)

Total de arquivos encontrados: 400
Arquivos identificados:
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_CO_DF_A001_BRASILIA_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_CO_GO_A002_GOIANIA_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_CO_MS_A702_CAMPO GRANDE_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_CO_MT_A901_CUIABA_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_NE_AL_A303_MACEIO_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_NE_BA_A401_SALVADOR_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_NE_CE_A305_FORTALEZA_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/INMET_PADRONIZADO/2008/INMET_NE_MA_A203_SAO LUIS_01-01-2008_A_31-12-2008.CSV
/content/drive/MyDrive/Leptospirose/I

In [38]:
# Ver nomes dos arquivos para entender o padrão
print(" Arquivos encontrados:")
for arquivo in arquivos_clima:
    nome = os.path.basename(arquivo)
    print(f"  - {nome}")
print("\n")

 Arquivos encontrados:
  - INMET_CO_DF_A001_BRASILIA_01-01-2008_A_31-12-2008.CSV
  - INMET_CO_GO_A002_GOIANIA_01-01-2008_A_31-12-2008.CSV
  - INMET_CO_MS_A702_CAMPO GRANDE_01-01-2008_A_31-12-2008.CSV
  - INMET_CO_MT_A901_CUIABA_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_AL_A303_MACEIO_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_BA_A401_SALVADOR_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_CE_A305_FORTALEZA_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_MA_A203_SAO LUIS_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_PB_A320_JOAO PESSOA_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_PE_A301_RECIFE_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_PI_A312_TERESINA_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_RN_A304_NATAL_01-01-2008_A_31-12-2008.CSV
  - INMET_NE_SE_A409_ARACAJU_01-01-2008_A_31-12-2008.CSV
  - INMET_N_AC_A104_RIO BRANCO_11-07-2008_A_31-12-2008.CSV
  - INMET_N_AM_A101_MANAUS_01-01-2008_A_31-12-2008.CSV
  - INMET_N_PA_A201_BELEM_01-01-2008_A_31-12-2008.CSV
  - INMET_N_RO_A925_PORTO VELHO_01-01-2008_A_31-12-2008.CS

In [39]:
lista_clima = []

mapa_meses = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

print("Processando arquivos...\n")

for arquivo in arquivos_clima:
    nome = os.path.basename(arquivo).replace('.CSV', '').replace('.csv', '')
    partes = nome.split('_')

    estado = None
    ano = None

    # DETECTAR PADRÃO
    if len(partes) == 3:
        estado = partes[1]
        try:
            ano = int(partes[2])
        except ValueError:
            print(f"  PULANDO (ano invalido): {nome}")
            continue

    elif len(partes) >= 7:
        estado = partes[2]
        data_parte = partes[5] if len(partes) > 5 else None
        if data_parte:
            match = re.search(r'(\d{4})', data_parte)
            if match:
                ano = int(match.group(1))
    else:
        print(f"  PULANDO (formato desconhecido): {nome}")
        continue

    # Validar
    if not estado or not ano or ano < 2000 or ano > 2030:
        print(f"  PULANDO: {nome} (Estado: {estado}, Ano: {ano})")
        continue

    print(f"Processando: {estado} - {ano}... ", end="")

    # Ler base (agora com decimal='.')
    try:
        df = pd.read_csv(
            arquivo,
            sep=';',
            encoding='latin1',
        )
    except Exception as e:
        print(f"Erro ao ler: {e}")
        continue

    # Verificar colunas
    colunas_necessarias = ['DATA', 'TEMPERATURA', 'UMIDADE', 'PRECIPITACAO']
    faltando = [col for col in colunas_necessarias if col not in df.columns]

    if faltando:
        print(f"Faltando: {faltando}")
        continue

    # GARANTIR que sao numericas
    df['TEMPERATURA'] = pd.to_numeric(df['TEMPERATURA'], errors='coerce')
    df['UMIDADE'] = pd.to_numeric(df['UMIDADE'], errors='coerce')
    df['PRECIPITACAO'] = pd.to_numeric(df['PRECIPITACAO'], errors='coerce')

    # Adicionar contexto
    df["ESTADO"] = estado
    df["ANO"] = ano

    # Converter DATA
    df["DATA"] = pd.to_datetime(df["DATA"], errors="coerce")
    df = df.dropna(subset=["DATA"])

    if len(df) == 0:
        print(f"Sem dados validos")
        continue

    # Criar mes
    df["MES"] = df["DATA"].dt.month

    # Remover linhas com valores numericos faltantes antes de agregar
    df = df.dropna(subset=['TEMPERATURA', 'UMIDADE', 'PRECIPITACAO'], how='all')

    # Agregar por mes
    df_mensal = (
        df.groupby(["ANO", "MES", "ESTADO"], as_index=False)
          .agg({
              "TEMPERATURA": "mean",
              "UMIDADE": "mean",
              "PRECIPITACAO": "sum"
          })
    )

    lista_clima.append(df_mensal)
    print(f"OK - {len(df_mensal)} meses")

# Concatenar
if lista_clima:
    clima = pd.concat(lista_clima, ignore_index=True)
    clima["MES"] = clima["MES"].map(mapa_meses)

    print(f"\n{'='*80}")
    print(f"PROCESSAMENTO CONCLUIDO!")
    print(f"{'='*80}")
    print(f"Total: {len(clima)} registros")
    print(f"Anos: {sorted(clima['ANO'].unique())}")
    print(f"Estados ({len(clima['ESTADO'].unique())}): {sorted(clima['ESTADO'].unique())}")

else:
    print("\nNenhum arquivo processado!")

Processando arquivos...

Processando: DF - 2008... OK - 12 meses
Processando: GO - 2008... OK - 12 meses
Processando: MS - 2008... OK - 12 meses
Processando: MT - 2008... OK - 12 meses
Processando: AL - 2008... OK - 4 meses
Processando: BA - 2008... OK - 9 meses
Processando: CE - 2008... OK - 4 meses
Processando: MA - 2008... OK - 12 meses
Processando: PB - 2008... OK - 12 meses
Processando: PE - 2008... OK - 8 meses
Processando: PI - 2008... OK - 12 meses
Processando: RN - 2008... OK - 12 meses
Processando: SE - 2008... OK - 10 meses
Processando: AC - 2008... OK - 6 meses
Processando: AM - 2008... OK - 9 meses
Processando: PA - 2008... OK - 12 meses
Processando: RO - 2008... OK - 12 meses
Processando: TO - 2008... OK - 12 meses
Processando: ES - 2008... OK - 12 meses
Processando: MG - 2008... OK - 12 meses
Processando: RJ - 2008... OK - 12 meses
Processando: SP - 2008... OK - 12 meses
Processando: PR - 2008... OK - 12 meses
Processando: RS - 2008... OK - 12 meses
Processando: SC - 200

# ***JUNÇÃO DE TABELAS***

In [40]:
base_final = pd.merge(casos, clima, on=['ANO','MES','ESTADO'], how='inner')
base_final

,MES,ESTADO,CASOS,ANO,TEMPERATURA,UMIDADE,PRECIPITACAO
0,Janeiro,RO,0,2010,26.042645,85.914980,18.4
1,Fevereiro,RO,0,2010,26.566667,84.866071,148.0
2,Abril,RO,1,2010,26.912639,81.312500,67.2
3,Maio,RO,3,2010,26.007661,80.384409,86.2
4,Junho,RO,1,2010,26.273296,75.956885,28.8
...,...,...,...,...,...,...,...
4168,Agosto,DF,3,2009,20.922984,55.614247,17.2
4169,Setembro,DF,3,2009,22.791528,63.197222,47.4
4170,Outubro,DF,1,2009,21.822423,76.303621,272.6
4171,Novembro,DF,2,2009,22.338194,74.454167,191.2


In [41]:
base_final.to_csv("/content/drive/MyDrive/Leptospirose/base_integrada_leptospirose_2008-2023.csv", index=False, sep=';')
print("\n💾 Arquivo consolidado salvo com sucesso!")


💾 Arquivo consolidado salvo com sucesso!
